In [ ]:
import pandas as pd
from collections import Counter

In [ ]:
df = pd.read_json("multipref_human_vs_gpt4_overall.jsonl", lines=True)
tags = pd.read_json("multipref_tags.jsonl", lines=True)

In [ ]:
df = df.merge(tags[["comparison_id", "tags"]], how="left", on="comparison_id")

In [ ]:
df = pd.concat([df.drop(columns=["tags"]), pd.json_normalize(df["tags"])], axis=1)

In [ ]:
df.to_json(
    "to_upload/multipref_human_vs_gpt4_overall.jsonl", lines=True, orient="records"
)

In [ ]:
# Only get overall preference
def get_majority(row, preference="overall_pref", binarize=True):
    normal_workers = [i["overall_pref"] for i in row["normal_worker_annotations"]]
    expert_workers = [i["overall_pref"] for i in row["expert_worker_annotations"]]
    all_annotations = normal_workers + expert_workers

    if binarize:
        final_annotations = []
        for annotation in all_annotations:
            if annotation == "A-is-slightly-better":
                annotation = "A-is-clearly-better"
            elif annotation == "B-is-slightly-better":
                annotation = "B-is-clearly-better"
            else:
                annotation = annotation
            final_annotations.append(annotation)
    else:
        final_annotations = all_annotations

    return Counter(final_annotations).most_common()[0][0]


# Use majority preference as main human preference
df["pref_human"] = df.apply(lambda x: get_majority(x), axis=1)

In [ ]:
df.sort_values(by="prompt_hash")

In [ ]:
gpt4_df = pd.read_json("../../data/OE-Adapt-GPT4-Overall.jsonl", lines=True).rename(
    columns={"pref": "pref_gpt4"}
)

In [ ]:
df = df.merge(
    gpt4_df[["comparison_id", "pref_gpt4", "evaluator"]], on="comparison_id", how="left"
)

In [ ]:
df.to_json("multipref_human_vs_gpt4_overall.jsonl", lines=True, orient="records")